# UBER : 

### Description de l'entreprise

Uber est l'une des startups les plus connues au monde. Cela a commencé comme une application de covoiturage pour les personnes qui ne pouvaient pas se permettre un taxi. Désormais, Uber a étendu ses activités à la livraison de nourriture avec Uber Eats , à la livraison de colis, au transport de marchandises et même au transport urbain avec Jump Bike et Lime que l'entreprise a financés.

L'objectif de l'entreprise est de révolutionner le transport à travers le monde. Elle opère aujourd'hui sur environ 70 pays et 900 villes et génère plus de 14 milliards de dollars de chiffre d'affaires !

### Projet

L'un des principaux problèmes rencontrés par l'équipe d'Uber est que parfois les chauffeurs ne sont pas là lorsque les utilisateurs en ont besoin.

Et les recherches d'Uber montrent que les utilisateurs acceptent d'attendre 5 à 7 minutes, sinon ils annuleraient leur trajet.

Par conséquent, l'équipe de données d'Uber aimerait travailler sur un projet dans lequel son application recommanderait des zones chaudes dans les grandes villes à n'importe quel moment de la journée.

### Objectifs

Uber dispose déjà de données sur les ramassages dans les grandes villes. Votre objectif est de créer des algorithmes qui détermineront où se trouvent les zones sensibles dans lesquelles les conducteurs doivent se trouver. Par conséquent, vous :

- Créer un algorithme pour trouver les zones chaudes
- Visualisez les résultats sur un joli tableau de bord

NB : la recherche sera portée uniquement sur la ville de New_York

### I - Création de l'algorithme (avec la méthode Kmeans) : 

#### A - Analyse du jeu de données (sur 1 fichier): 

In [2]:
#Importer les librairies : 
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
pio.renderers.default = 'notebook_connected'
pio.renderers.default = "iframe_connected"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install plotly==4.9.0
!pip install jupyterlab "ipywidgets>=7.5"
!jupyter labextension install jupyterlab-plotly@4.9.0
!jupyter labextension install @jupyter-widgets/jupyterlab-manager plotlywidget@4.9.0

In [4]:
!pip install zipfile36

In [5]:
#Ouvrir le fichier ZIP et récupérer tout les fichiers CSV
from zipfile import ZipFile

zip_file = ZipFile('/content/gdrive/MyDrive/JEDHA/Minis_projets/Uber/uber_trip_data.zip')
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}

In [6]:
print(dfs)

{'uber-trip-data/taxi-zone-lookup.csv':      LocationID        Borough                     Zone
0             1            EWR           Newark Airport
1             2         Queens              Jamaica Bay
2             3          Bronx  Allerton/Pelham Gardens
3             4      Manhattan            Alphabet City
4             5  Staten Island            Arden Heights
..          ...            ...                      ...
260         261      Manhattan       World Trade Center
261         262      Manhattan           Yorkville East
262         263      Manhattan           Yorkville West
263         264        Unknown                  Unknown
264         265        Unknown                  Unknown

[265 rows x 3 columns], '__MACOSX/uber-trip-data/._taxi-zone-lookup.csv': Empty DataFrame
Columns: [Unnamed: 0]
Index: [], 'uber-trip-data/uber-raw-data-apr14.csv':                  Date/Time      Lat      Lon    Base
0         4/1/2014 0:11:00  40.7690 -73.9549  B02512
1         4/1/20

In [7]:
#On va faire le projet sur le fichier d'avril 2014
dfs['uber-trip-data/uber-raw-data-apr14.csv']

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512
...,...,...,...,...
564511,4/30/2014 23:22:00,40.7640,-73.9744,B02764
564512,4/30/2014 23:26:00,40.7629,-73.9672,B02764
564513,4/30/2014 23:31:00,40.7443,-73.9889,B02764
564514,4/30/2014 23:32:00,40.6756,-73.9405,B02764


In [8]:
#Convertir le jeu de données en dataframe 
df=pd.DataFrame(dfs['uber-trip-data/uber-raw-data-apr14.csv'])

In [9]:
df.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


In [10]:
#Voir le nombre de lignes et de colonnes dans le dataframe 
df.shape

(564516, 4)

In [11]:
#En raison des lenteurs sur les outils, on va réduire le dataset : 
df = df.sample(10000)

#### Séparer et convertir les données Date/Time

In [12]:
#Contrôler le type de la colonne "Date/Time"
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 382456 to 453992
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date/Time  10000 non-null  object 
 1   Lat        10000 non-null  float64
 2   Lon        10000 non-null  float64
 3   Base       10000 non-null  object 
dtypes: float64(2), object(2)
memory usage: 390.6+ KB


In [13]:
#On va convertir la colonne "Date/Time" au format datetime + séparer les données dans deux nouvelles colonnes 
df['Date'] = pd.to_datetime(df['Date/Time']).dt.date
df['Time'] = pd.to_datetime(df['Date/Time']).dt.time

In [14]:
#On va jeter la colonne "Date/Time"
df=df.drop(["Date/Time"], axis=1)

In [15]:
#On contrôle la mise à jour du dataframe 
df.tail()

,Lat,Lon,Base,Date,Time
85230,40.7131,-73.9851,B02598,2014-04-10,19:24:00
515242,40.7662,-73.9793,B02682,2014-04-26,12:53:00
174739,40.7547,-73.9702,B02598,2014-04-25,22:23:00
536285,40.7266,-74.0074,B02682,2014-04-29,15:09:00
453992,40.7494,-73.9724,B02682,2014-04-17,18:37:00


On aura besoin des données dans date et time, pour produire le graphique, pour cela on doit convertir les données en numérique

In [16]:
#Convertir la colonnes Date en nombre : 
df['Date'] = pd.to_numeric(pd.to_datetime(df['Date']))

In [17]:
#En ce qui concerne la colonne Time, on a du séparer les heures et les minutes pour les convertir en numérique 
df["hour"] = df.Time.apply(lambda x: x.hour)
df["minute"] = df.Time.apply(lambda x: x.minute)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 382456 to 453992
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     10000 non-null  float64
 1   Lon     10000 non-null  float64
 2   Base    10000 non-null  object 
 3   Date    10000 non-null  int64  
 4   Time    10000 non-null  object 
 5   hour    10000 non-null  int64  
 6   minute  10000 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 625.0+ KB


In [19]:
#On va jeter la colonne Time
df=df.drop(["Time"], axis=1)

In [20]:
df.tail()

,Lat,Lon,Base,Date,hour,minute
85230,40.7131,-73.9851,B02598,1397088000000000000,19,24
515242,40.7662,-73.9793,B02682,1398470400000000000,12,53
174739,40.7547,-73.9702,B02598,1398384000000000000,22,23
536285,40.7266,-74.0074,B02682,1398729600000000000,15,9
453992,40.7494,-73.9724,B02682,1397692800000000000,18,37


#### Voir combien de différentes données on a dans la colonne Base

In [21]:
df['Base'].value_counts()

B02682    4045
B02598    3259
B02617    1901
B02512     638
B02764     157
Name: Base, dtype: int64

Contrôler si on des valeurs manquantes :

In [22]:
print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Percentage of missing values: 


Lat       0.0
Lon       0.0
Base      0.0
Date      0.0
hour      0.0
minute    0.0
dtype: float64

Normalize dataset

In [23]:
from sklearn.preprocessing import  OneHotEncoder, StandardScaler

In [24]:
from sklearn.compose import ColumnTransformer

In [25]:
#On standardise les données numériques
numeric_features = [0, 1] # Positions des colonnes quantitatives des colonnes 'lat' et 'lng'
numeric_transformer = StandardScaler()

# Création du transformer pour les variables catégorielles
categorical_features = [2] # On a encoder la colonne 'Base'
categorical_transformer = OneHotEncoder(drop='first')

# On combine les transformers dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocessings sur le dataset
print(df.head())
X = preprocessor.fit_transform(df) 
print('...Terminé.')
print(X[0:5, :])
print()

            Lat      Lon    Base                 Date  hour  minute
382456  40.7558 -73.9834  B02682  1396828800000000000    19      14
188231  40.7628 -73.9820  B02598  1398556800000000000    13      58
190023  40.7633 -73.9649  B02598  1398556800000000000    18      58
390178  40.7551 -73.9765  B02682  1396915200000000000    20      19
135524  40.7495 -73.9797  B02598  1397865600000000000    20       9
...Terminé.
[[ 0.42634257 -0.12928209  0.          0.          1.          0.        ]
 [ 0.62296421 -0.10109474  1.          0.          0.          0.        ]
 [ 0.63700861  0.24319362  1.          0.          0.          0.        ]
 [ 0.40668041  0.00964128  0.          0.          1.          0.        ]
 [ 0.24938309 -0.05478695  1.          0.          0.          0.        ]]



Select  K  optimal clusters

Méthode Elbow

In [26]:
from sklearn.cluster import KMeans

In [27]:
# Let's create a loop that will collect the Within-sum-of-square (wcss) for each value K 
# Let's use .inertia_ parameter to get the within sum of square value for each value K 
wcss =  []
k = []
for i in range (1,11): 
    kmeans = KMeans(n_clusters= i, init = "k-means++", random_state = 0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    k.append(i)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))
    
#Avec la méthode elbow ont veut trouver la variance (=inertie) des observations, mais on veut le moins de variance possible et pas un trop gros nombre de cluster ! 
#C'est à dire on ne veut pas une seule oservation par cluster (ex : 98 cluster pour 100 observation = BOF)

WCSS for K=1 --> 26299.844399999994
WCSS for K=2 --> 20595.556358132966
WCSS for K=3 --> 15945.350001386458
WCSS for K=4 --> 13918.981195633472
WCSS for K=5 --> 12174.97250265864
WCSS for K=6 --> 10508.811701136576
WCSS for K=7 --> 9330.634438334873
WCSS for K=8 --> 7981.145764065683
WCSS for K=9 --> 7177.795467277367
WCSS for K=10 --> 6407.69258237129


In [28]:
!pip install plotly -q

In [29]:
!pip install tornado -q

In [30]:
!pip install plotly --upgrade

  Using cached plotly-5.5.0-py2.py3-none-any.whl (26.5 MB)
  Attempting uninstall: plotly
    Found existing installation: plotly 4.9.0
    Uninstalling plotly-4.9.0:
      Successfully uninstalled plotly-4.9.0


In [31]:
# Let's visualize using plotly
import plotly.express as px

In [32]:
# Create DataFrame
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

# Render
fig.show(renderer="notebook")
#fig.show(renderer="iframe") # if using workspace

Output hidden; open in https://colab.research.google.com to view.

D'après la méthode Elbow, on devrait choisir un nombre de 4 clusters
On pourrait confirmer le nombre de cluster avec la méthode Silhouette, mais ça n'arrive pas à sortir les résultats... 

## Apply K-Means


In [33]:
# On ré-entraîne un KMeans avec le nombre optimal de clusters
kmeans = KMeans(n_clusters= 4)
kmeans.fit(X)

KMeans(n_clusters=4)

In [34]:
df.loc[:,'Cluster_KMeans'] = kmeans.predict(X)
df.head()

,Lat,Lon,Base,Date,hour,minute,Cluster_KMeans
382456,40.7558,-73.9834,B02682,1396828800000000000,19,14,0
188231,40.7628,-73.9820,B02598,1398556800000000000,13,58,1
190023,40.7633,-73.9649,B02598,1398556800000000000,18,58,1
390178,40.7551,-73.9765,B02682,1396915200000000000,20,19,0
135524,40.7495,-73.9797,B02598,1397865600000000000,20,9,3


In [35]:
fig = px.scatter_mapbox(
        data_frame=df, 
        lat="Lat", 
        lon="Lon",
        color="Cluster_KMeans",
        size="Cluster_KMeans",
        mapbox_style="carto-positron"
)

fig.show(renderer="notebook")

Output hidden; open in https://colab.research.google.com to view.

## Apply DBSCAN

In [36]:
# importer les librairies
from sklearn.cluster import DBSCAN
import numpy as np

In [46]:
# Instanciate DBSCAN 
db = DBSCAN(eps=0.5, min_samples=100, metric="manhattan")   #100 au min_samples permet de ne pas avoir trop de clusters
db.fit(X)

DBSCAN(metric='manhattan', min_samples=100)

In [47]:
#On créé une nouvelle colonne Cluster_dbscan pour avoir un comparatif avec Kmeans
df.loc[:,'Cluster_DBSCAN'] = db.fit_predict(X)
df.head()

,Lat,Lon,Base,Date,hour,minute,Cluster_KMeans,Cluster_DBSCAN
382456,40.7558,-73.9834,B02682,1396828800000000000,19,14,0,0
188231,40.7628,-73.9820,B02598,1398556800000000000,13,58,1,1
190023,40.7633,-73.9649,B02598,1398556800000000000,18,58,1,1
390178,40.7551,-73.9765,B02682,1396915200000000000,20,19,0,0
135524,40.7495,-73.9797,B02598,1397865600000000000,20,9,3,1


In [48]:
#Pour savoir les nombre de clusters déterminés par DBSCAN : 
np.unique(db.labels_)

#On n'en a 4 aussi et attention le -1 est à exclure car il represente le bruit/noise du modèle, car les données qui n'ont pas pu être classifiées.

array([-1,  0,  1,  2,  3])

In [49]:
#Graphique pour voir le résultat avec DSCAN 
fig = px.scatter_mapbox(
        data_frame=df[df.Cluster_DBSCAN != -1],   #On doit exclure le -1 dans la colonne "Cluster_DBSCAN"
        lat="Lat", 
        lon="Lon",
        color="Cluster_KMeans",
        size="Cluster_KMeans",
        mapbox_style="carto-positron"
)

fig.show(renderer="notebook")

Output hidden; open in https://colab.research.google.com to view.